<a href="https://colab.research.google.com/github/Erickrus/leetcode/blob/master/node2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install git+https://github.com/eliorc/node2vec
!pip3 install networkx
!pip install paramiko

In [0]:
import gensim, node2vec
import networkx as nx
import node2vec.edges

In [0]:
# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
n2v = node2vec.Node2Vec(
    graph, 
    dimensions=64, 
    walk_length=30, 
    num_walks=200, 
    workers=4
)  # Use temp_folder for big graphs

# Embed nodes
model = n2v.fit(
    window=10, 
    min_count=1, 
    batch_words=4
)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

# Look for most similar nodes
model.wv.most_similar('2')  # Output node names are always strings


In [0]:
EMBEDDING_FILENAME = "/tmp/emb"
EMBEDDING_MODEL_FILENAME = "/tmp/emb_model"
EDGES_EMBEDDING_FILENAME = "/tmp/emb_edge"

# Save embeddings for later use
model.wv.save_word2vec_format(EMBEDDING_FILENAME)

# Save model for later use
model.save(EMBEDDING_MODEL_FILENAME)

edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)

# Look for embeddings on the fly - here we pass normal tuples
print(edges_embs[('1', '2')])
''' OUTPUT
array([ 5.75068220e-03, -1.10937878e-02,  3.76693785e-01,  2.69105062e-02,
       ... ... ....
       ..................................................................],
      dtype=float32)
'''

# Get all edges in a separate KeyedVectors instance - use with caution could be huge for big networks
edges_kv = edges_embs.as_keyed_vectors()

# Look for most similar edges - this time tuples must be sorted and as str
edges_kv.most_similar(str(('1', '2')))

# Save embeddings for later use
edges_kv.save_word2vec_format(EDGES_EMBEDDING_FILENAME)

In [0]:
!ls -al /tmp/emb*

-rw-r--r-- 1 root root   72901 Apr 12 05:33 /tmp/emb
-rw-r--r-- 1 root root 6770639 Apr 12 05:33 /tmp/emb_edge
-rw-r--r-- 1 root root   88252 Apr 12 05:33 /tmp/emb_model
